#### ✅ **제목 전처리 외 처리 과정**

**[CONTENTS & VOD]**

1. disp_rtm_min(object -> 초단위로 변경)
2. 모든 컬럼이 동일한 행 제거
3. cont_log_dt 에서 날짜 ('cont_log_date’), 시간대 ('cont_log_hour’), 요일(’strt_dat_of_week’) 추출해서 추가
4. (VOD) subsr,'vod_strip', strt_date 별로 use_tms 합 컬럼 추가
5. (VOD) sebsr 별 상세보기 클릭 횟수(CONT) 를 ‘cont_count’ 컬럼 추가

**[이상치 제거]**

1. 모든 컬럼이 동일한 행 제거
2. 일별 총 시청시간 합 (use_tms)이 5분 이하인 경우는 이상치로 간주하고 삭제
3. content에는 존재하지만 vod에는 존재하지 않는 행 *→ 일단 보류*

**[평가기준]**
- 각 VOD 러닝타임에 따른 use_tms 비중의 합 (total_user_tms)으로 진행함
- 상세보기(cont)은 개수 변수로 추가함

- 클릭율, use_tms 활용한 평가 지표 고려



**[FM 모델 결과]**
- MSE : 3.74 (러닝타임 별 시청시간 고려 전)
- MSE : 3.2611(러닝타임 별 시청시간 고려 적용)
- 우선은 total_use_tms 예측하는 것으로 구현함
- total_use_tms 에 따라 10월 시청 여부로 바꾸는 것도 고려중


[+추가 고려사항]
- 평가기준 설정
- disp_rtm : 1분이나 2분은 광고나 예고편으로 간주 -> 제거하는게 좋을지
- 추천 모델 library 적용 : pip librecommender
- 이외 새로운 추천 모델 생각해보기
  - ex) 시간대별 추천 리스트 (멜론) / 날씨 별 추천 VOD
  - ex) 시청 시간대별 특징 파악


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import re
from itertools import chain
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# 데이터 읽기
vod_aug = pd.read_csv('/content/drive/MyDrive/dxdata/데이터스쿨_3차_VOD_2308.csv', sep="\t", encoding= 'cp949')
vod_sep = pd.read_csv('/content/drive/MyDrive/dxdata/데이터스쿨_3차_VOD_2309.csv', sep="\t", encoding= 'cp949')
cont_aug = pd.read_csv('/content/drive/MyDrive/dxdata/데이터스쿨_3차_콘텐츠_2308.csv', sep="\t", encoding= 'cp949')
cont_sep = pd.read_csv('/content/drive/MyDrive/dxdata/데이터스쿨_3차_콘텐츠_2309.csv', sep="\t", encoding= 'cp949')

columns = ['subsr', 'chnl_nm', 'in_dt', 'use_tms', 'pgm_nm',  'summary',   'cast',   'program_duration_minutes']
channel_aug = pd.read_csv('/content/drive/MyDrive/dxdata/데이터스쿨3차_채널PGM_2308월.csv', encoding = 'CP949', usecols= columns)
channel_sep = pd.read_csv('/content/drive/MyDrive/dxdata/데이터스쿨3차_채널PGM_2309월.csv', encoding='cp949', usecols= columns)

<ipython-input-3-b4a1e256b96a>:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  channel_aug = pd.read_csv('/content/drive/MyDrive/dxdata/데이터스쿨3차_채널PGM_2308월.csv', encoding = 'CP949', usecols= columns)
<ipython-input-3-b4a1e256b96a>:9: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  channel_sep = pd.read_csv('/content/drive/MyDrive/dxdata/데이터스쿨3차_채널PGM_2309월.csv', encoding='cp949', usecols= columns)


In [4]:
vod_aug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5615 entries, 0 to 5614
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           5615 non-null   int64 
 1   asset_nm        5615 non-null   object
 2   ct_cl           5615 non-null   object
 3   genre_of_ct_cl  5615 non-null   object
 4   use_tms         5615 non-null   int64 
 5   SMRY            5612 non-null   object
 6   ACTR_DISP       5612 non-null   object
 7   disp_rtm        5615 non-null   object
 8   strt_dt         5615 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 394.9+ KB


In [5]:
cont_aug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15677 entries, 0 to 15676
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           15677 non-null  int64 
 1   series_nm       7516 non-null   object
 2   super_asset_nm  15677 non-null  object
 3   ct_cl           15677 non-null  object
 4   genre_of_ct_cl  15677 non-null  object
 5   SMRY            15677 non-null  object
 6   ACTR_DISP       15677 non-null  object
 7   disp_rtm        14512 non-null  object
 8   log_dt          15677 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.1+ MB


#### ✅ [제목 전처리]
=> 새로운 데이터 맞춰서 추후 반영 예정

In [6]:
#전처리 함수

#pinfo(괄호 데이터) 추출
def get_pinfo(li):
    pattern = re.compile('\(([^)]+)')
    return [pattern.findall(str(i)) for i in li]

#pinfo(괄호 데이터) 제거
def remove_pinfo(li):
    pattern = r'\([^)]*\)'
    temp = []
    for i in li:
        tmp = re.sub(pattern, repl = '', string = str(i))
        temp = np.append(temp, tmp)
    return temp

#괄호 안의 업로드 날짜 추출
def get_upload_date(li):
    pattern = re.compile('\d+/\d+/\d+')
    return [pattern.findall(str(i)) for i in li]

#괄호 안의 업로드 날짜 제거
def remove_upload_date(li):
    pattern = r'\d+/\d+/\d+'
    temp = []
    for i in li:
        tmp = re.sub(pattern, repl = '', string = str(i))
        temp.append(tmp)
    return temp

#회차 정보 제거
def remove_series(li):
    pattern = r'\s{0,1}\d{1,7}[회,화]\s{0,1}'
    temp = []
    for i in li:
        tmp = re.sub(pattern, repl = '', string = str(i))
        temp = np.append(temp, tmp)
    return temp

#공백 제거
def remove_space(li):
    temp = []
    for i in li:
        tmp = str(i).replace(" ", "")
        temp = np.append(temp, tmp)
    return temp

#맨 뒤에 . 붙어있는거 제거
def remove_punkt(li):
    temp = []
    for i in li:
        if i[-1] == '.':
            temp.append(i[:-1])
        else:
            temp.append(i)
    return temp

In [7]:
upload_date = get_upload_date(vod_aug.asset_nm.values)
upload_date = [date for sub_date in upload_date for date in (sub_date or [np.nan])]
vod_aug['upload_date'] = upload_date
vod_aug['upload_date'] = pd.to_datetime(vod_aug.upload_date, format = '%y/%m/%d')

In [8]:
#vod df에 추가할 vod 이름만 추출
vod_name = remove_upload_date(vod_aug.asset_nm.values)

vod_pinfo = get_pinfo(vod_name)
vod_aug['pinfo'] = vod_pinfo

# vod08['pinfo'] = vod08['pinfo'].str.get()
vod_aug['pinfo'] =  vod_aug['pinfo'].apply(lambda x: str(x).replace('[','').replace(']',''))

vod_pinfo = list(chain(*vod_pinfo))
vod_pinfo = pd.Series(vod_pinfo).unique()

vod_name = remove_pinfo(vod_name)
vod_name = remove_series(vod_name)


In [9]:
cont_name = cont_aug.super_asset_nm.fillna('X').values
cont_pinfo = get_pinfo(cont_name)

series_name = cont_aug.series_nm.fillna('X').values
series_pinfo = get_pinfo(series_name)

cont_pinfo_all = [list(set(sum([p1,p2], []))) for p1, p2 in zip(cont_pinfo, series_pinfo)]
cont_aug['pinfo'] = cont_pinfo_all
cont_aug['pinfo'] = cont_aug['pinfo'].apply(lambda x: str(x).replace('[', '').replace(']', ''))

cont_pinfo = list(chain(*cont_pinfo_all))
cont_pinfo = pd.Series(cont_pinfo).unique()

cont_name = remove_pinfo(cont_aug.super_asset_nm.values)
cont_name = remove_series(cont_name)

In [10]:
vod_name = remove_punkt(vod_name)
cont_name = remove_punkt(cont_name)

In [11]:
vod_aug['vod_name'] = vod_name
cont_aug['cont_name'] = cont_name

In [12]:
## 9월
upload_date = get_upload_date(vod_sep.asset_nm.values)
upload_date = [date for sub_date in upload_date for date in (sub_date or [np.nan])]
vod_sep['upload_date'] = upload_date
vod_sep['upload_date'] = pd.to_datetime(vod_sep.upload_date, format = '%y/%m/%d')

In [13]:
#vod df에 추가할 content 이름만 추출
vod_name2 = remove_upload_date(vod_sep.asset_nm.values)

vod_pinfo2 = get_pinfo(vod_name2)
vod_sep['pinfo'] = vod_pinfo2

# vod08['pinfo'] = vod08['pinfo'].str.get()
vod_sep['pinfo'] =  vod_sep['pinfo'].apply(lambda x: str(x).replace('[','').replace(']',''))

vod_pinfo2 = list(chain(*vod_pinfo2))
vod_pinfo2 = pd.Series(vod_pinfo2).unique()

vod_name2 = remove_pinfo(vod_name2)
vod_name2 = remove_series(vod_name2)
cont_name2 = cont_sep.super_asset_nm.fillna('X').values
cont_pinfo2 = get_pinfo(cont_name2)

series_name2 = cont_sep.series_nm.fillna('X').values
series_pinfo2 = get_pinfo(series_name2)

cont_pinfo_all2 = [list(set(sum([p1,p2], []))) for p1, p2 in zip(cont_pinfo2, series_pinfo2)]
cont_sep['pinfo'] = cont_pinfo_all2
cont_sep['pinfo'] = cont_sep['pinfo'].apply(lambda x: str(x).replace('[', '').replace(']', ''))

cont_pinfo2 = list(chain(*cont_pinfo_all2))
cont_pinfo2 = pd.Series(cont_pinfo2).unique()
cont_name2 = remove_pinfo(cont_sep.super_asset_nm.values)
cont_name2 = remove_series(cont_name2)
vod_name2 = remove_punkt(vod_name2)
cont_name2 = remove_punkt(cont_name2)

vod_sep['vod_name'] = vod_name2
cont_sep['cont_name'] = cont_name2

In [14]:
## 추가전처리
#8월 데이터

#백설 공주
#vod -> 백설 공주 / cont -> 백설공주 (55,56)

#내생애가장아름다운일주일
#vod -> 내 생애 가장 아름다운 일주일 / cont -> 내생애가장아름다운일주일 (9160 ~ 9165)

#vod쪽으로 통합 (space 있는 버전 )

cont_aug.loc[56, 'cont_name']= '백설 공주'
cont_aug.loc[8047, 'cont_name'] = '내 생애 가장 아름다운 일주일'
#9월 데이터

#링
# vod -> 링(space) / cont -> 링
#vod를 링으로 변경 (68)

#장화신은 고양이
#vod -> 장화 신은 고양이 / cont -> 장화신은 고양이
#vod를 장화신은 고양이로 변경 (google 검색 기준) (2830)

#vod -> 장화신은고양이 / cont -> 장화신은 고양이
#vod 변경 (google 기준) (3500, 3501)


#내 생애 가장 아름다운 일주일
#vod -> 내 생애 가장 아름다운 일주일 / cont -> 내생애가장아름다운일주일
#cont 변경 -> space 추가 (8044 - 8047)

vod_sep.loc[3501, 'vod_name']= '장화신은 고양이'

In [15]:
vod_strip = remove_space(vod_aug.vod_name.values)
cont_strip = remove_space(cont_aug.cont_name.values)

vod_strip2 = remove_space(vod_sep.vod_name.values)
cont_strip2 = remove_space(cont_sep.cont_name.values)
vod_aug['vod_strip'] = vod_strip
cont_aug['cont_strip'] = cont_strip

vod_sep['vod_strip'] = vod_strip2
cont_sep['cont_strip'] = cont_strip2

# 빈칸 제거
set1 = set(vod_aug.vod_strip.unique()).difference(set(cont_aug.cont_strip.unique())) # 183
li1 = list(set1)

set11 = set(vod_sep.vod_strip.unique()).difference(set(cont_sep.cont_strip.unique())) # 183
li11 = list(set11)
st1 = []
for i in li1:
    st1.append(vod_aug[vod_aug.vod_strip == i].index.tolist()[0])

st11 = []
for i in li11:
    st11.append(vod_sep[vod_sep.vod_strip == i].index.tolist()[0])
set2 = set(vod_aug.vod_name.unique()).difference(set(cont_aug.cont_name.unique())) # 185
li2 = list(set2)

set22 = set(vod_sep.vod_name.unique()).difference(set(cont_sep.cont_name.unique())) # 185
li22 = list(set22)
st2 = []
for i in li2:
    st2.append(vod_aug[vod_aug.vod_name == i].index.tolist()[0])

st22 = []
for i in li22:
    st22.append(vod_sep[vod_sep.vod_name == i].index.tolist()[0])
cont_sep[(cont_sep.super_asset_nm.notnull()) & (cont_sep.super_asset_nm.str.contains('내생애가장'))]
vod_sep[vod_sep.vod_name == '장화신은 고양이']
cont_sep[cont_sep.cont_name == '장화신은 고양이']

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,pinfo,cont_name,cont_strip
11084,66056000,NaN,장화신은 고양이,영화,애니메이션,본 콘텐츠는 평생소장 콘텐츠로 서비스 해지 시까지 시청이 가능합니다. 한때는 알아주...,"안토니오 반데라스,셀마 헤이엑,자흐 칼리피아나키스",NaN,20230918050931,,장화신은 고양이,장화신은고양이
11085,63086000,NaN,장화신은 고양이,영화,애니메이션,본 콘텐츠는 평생소장 콘텐츠로 서비스 해지 시까지 시청이 가능합니다. 한때는 알아주...,"안토니오 반데라스,셀마 헤이엑,자흐 칼리피아나키스",NaN,20230924160207,,장화신은 고양이,장화신은고양이
11086,63086000,NaN,장화신은 고양이,영화,애니메이션,본 콘텐츠는 평생소장 콘텐츠로 서비스 해지 시까지 시청이 가능합니다. 한때는 알아주...,"안토니오 반데라스,셀마 헤이엑,자흐 칼리피아나키스",NaN,20230924160126,,장화신은 고양이,장화신은고양이
11087,63086000,NaN,장화신은 고양이,영화,애니메이션,본 콘텐츠는 평생소장 콘텐츠로 서비스 해지 시까지 시청이 가능합니다. 한때는 알아주...,"안토니오 반데라스,셀마 헤이엑,자흐 칼리피아나키스",NaN,20230924160152,,장화신은 고양이,장화신은고양이
11088,63086000,NaN,장화신은 고양이,영화,애니메이션,본 콘텐츠는 평생소장 콘텐츠로 서비스 해지 시까지 시청이 가능합니다. 한때는 알아주...,"안토니오 반데라스,셀마 헤이엑,자흐 칼리피아나키스",NaN,20230924160201,,장화신은 고양이,장화신은고양이


In [16]:
li = cont_sep[(cont_sep.super_asset_nm.notnull()) & (cont_sep.super_asset_nm.str.contains('짱구는 못말려: 동물소환 닌자'))].index.tolist()
cont_sep[(cont_sep.super_asset_nm.notnull()) & (cont_sep.super_asset_nm.str.contains('짱구는 못말려: 동물소환 닌자'))]
li = vod_aug[(vod_aug.asset_nm.notnull()) & (vod_aug.asset_nm.str.contains('짱구는 못말려: 수수께끼! 꽃피는'))].index.tolist()
vod_aug[(vod_aug.asset_nm.notnull()) & (vod_aug.asset_nm.str.contains('펜트하우스2히든룸'))]
for i in li:
    cont_sep.loc[i, 'cont_name'] = '짱구는 못말려: 동물소환 닌자 배꼽수비대'
cont_sep.loc[39, 'cont_name'] = '해리포터와마법사의돌:매지컬무비모드'

#### [이외 전처리]
1. disp_rtm_min(object -> 초단위로 변경)
2. 모든 컬럼이 동일한 행 제거
3. 컬럼 추가

In [17]:
cont08 = cont_aug.copy()
cont09 = cont_sep.copy()
vod08 = vod_aug.copy()
vod09 = vod_sep.copy()
channel08 = channel_aug.copy()
channel09 = channel_sep.copy()

disp_rtm_min (초단위로 변경)

In [18]:
# disp_rtm_min(object -> 초단위로 변경)
from datetime import datetime

cont08['disp_rtm'] = pd.to_datetime(cont08['disp_rtm'], format='%H:%M').dt.time # datetime 형식으로 변환
cont08['disp_rtm_sec']= cont08['disp_rtm'].apply(lambda x: x.hour * 3600 + x.minute*60)
cont09['disp_rtm'] = pd.to_datetime(cont09['disp_rtm'], format='%H:%M').dt.time # datetime 형식으로 변환
cont09['disp_rtm_sec']= cont09['disp_rtm'].apply(lambda x: x.hour * 3600 + x.minute*60)

In [19]:
vod08['disp_rtm'] = pd.to_datetime(vod08['disp_rtm'], format='%H:%M').dt.time # datetime 형식으로 변환
vod08['disp_rtm_sec']= vod08['disp_rtm'].apply(lambda x: x.hour * 3600 + x.minute*60)
vod09['disp_rtm'] = pd.to_datetime(vod09['disp_rtm'], format='%H:%M').dt.time # datetime 형식으로 변환
vod09['disp_rtm_sec']= vod09['disp_rtm'].apply(lambda x: x.hour * 3600 + x.minute*60)

In [20]:
vod09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5042 entries, 0 to 5041
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subsr           5042 non-null   int64         
 1   asset_nm        5042 non-null   object        
 2   ct_cl           5042 non-null   object        
 3   genre_of_ct_cl  5042 non-null   object        
 4   use_tms         5042 non-null   int64         
 5   SMRY            5041 non-null   object        
 6   ACTR_DISP       5041 non-null   object        
 7   disp_rtm        5042 non-null   object        
 8   strt_dt         5042 non-null   int64         
 9   upload_date     2039 non-null   datetime64[ns]
 10  pinfo           5042 non-null   object        
 11  vod_name        5042 non-null   object        
 12  vod_strip       5042 non-null   object        
 13  disp_rtm_sec    5042 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(9)
memory usage: 

In [21]:
cont08['log_dt'] = cont08['log_dt'].astype(str)
cont09['log_dt'] = cont09['log_dt'].astype(str)

# 날짜 부분 추출 및 datetime 형식으로 변환
cont08['cont_log_date'] = pd.to_datetime(cont08['log_dt'].str[:8], format='%Y%m%d', errors='coerce')
cont09['cont_log_date'] = pd.to_datetime(cont09['log_dt'].str[:8], format='%Y%m%d', errors='coerce')

# Hour
cont08['cont_log_hour'] = cont08['log_dt'].str[8:10].astype(int)
cont09['cont_log_hour'] = cont09['log_dt'].str[8:10].astype(int)

In [22]:
vod08['strt_dt'] = vod08['strt_dt'].astype(str)
vod09['strt_dt'] = vod09['strt_dt'].astype(str)

# 날짜 부분 추출 및 datetime 형식으로 변환
vod08['strt_date'] = pd.to_datetime(vod08['strt_dt'].str[:8], format='%Y%m%d', errors='coerce')
vod09['strt_date'] = pd.to_datetime(vod09['strt_dt'].str[:8], format='%Y%m%d', errors='coerce')

# Hour
vod08['strt_hour'] = vod08['strt_dt'].str[8:10].astype(int)
vod09['strt_hour'] = vod09['strt_dt'].str[8:10].astype(int)

In [23]:
def extract_datetime_info(df, date_column_name):
#     df[date_column_name] = df[date_column_name].astype(str)

#     df['year'] = df[date_column_name].str[:4]
#     df['minute'] = df[date_column_name].str[10:12]
#     df['sec'] = df[date_column_name].str[12:14]
#     df['month'] = df[date_column_name].str[4:6]
#     df['date'] = df[date_column_name].str[6:8]
#     df['hour'] = df[date_column_name].str[8:10]


#     df[date_column_name] = df[date_column_name].str[:8]
#     df[date_column_name] = pd.to_datetime(df[date_column_name], format='%Y%m%d')
    df['strt_day_of_week'] = df[date_column_name].dt.strftime('%A')
    return df

cont08 = extract_datetime_info(cont08, 'cont_log_date')
cont09 = extract_datetime_info(cont09, 'cont_log_date')

vod08 = extract_datetime_info(vod08, 'strt_date')
vod09 = extract_datetime_info(vod09, 'strt_date')

모든 컬럼이 동일한 행 제거

In [24]:
def remove_duplicate(df):
    df = df.drop_duplicates(keep='first')
    print("남아있는 중복행 :", df[df.duplicated(keep=False)])

In [25]:
remove_duplicate(cont08)
remove_duplicate(cont09)

남아있는 중복행 : Empty DataFrame
Columns: [subsr, series_nm, super_asset_nm, ct_cl, genre_of_ct_cl, SMRY, ACTR_DISP, disp_rtm, log_dt, pinfo, cont_name, cont_strip, disp_rtm_sec, cont_log_date, cont_log_hour, strt_day_of_week]
Index: []
남아있는 중복행 : Empty DataFrame
Columns: [subsr, series_nm, super_asset_nm, ct_cl, genre_of_ct_cl, SMRY, ACTR_DISP, disp_rtm, log_dt, pinfo, cont_name, cont_strip, disp_rtm_sec, cont_log_date, cont_log_hour, strt_day_of_week]
Index: []


In [26]:
remove_duplicate(vod08)
remove_duplicate(vod09)

남아있는 중복행 : Empty DataFrame
Columns: [subsr, asset_nm, ct_cl, genre_of_ct_cl, use_tms, SMRY, ACTR_DISP, disp_rtm, strt_dt, upload_date, pinfo, vod_name, vod_strip, disp_rtm_sec, strt_date, strt_hour, strt_day_of_week]
Index: []
남아있는 중복행 : Empty DataFrame
Columns: [subsr, asset_nm, ct_cl, genre_of_ct_cl, use_tms, SMRY, ACTR_DISP, disp_rtm, strt_dt, upload_date, pinfo, vod_name, vod_strip, disp_rtm_sec, strt_date, strt_hour, strt_day_of_week]
Index: []


#### 모델 적용을 위한 처리 과정

In [27]:
cont08_1 = cont08[['subsr', 'cont_strip', 'ct_cl', 'genre_of_ct_cl',
        'disp_rtm_sec', 'cont_log_date', 'cont_log_hour','strt_day_of_week']]
cont09_1 = cont09[['subsr', 'cont_strip', 'ct_cl', 'genre_of_ct_cl',
        'disp_rtm_sec', 'cont_log_date', 'cont_log_hour','strt_day_of_week']]

In [28]:
vod08_1 = vod08[['subsr', 'vod_strip', 'ct_cl', 'genre_of_ct_cl', 'use_tms',
        'disp_rtm_sec', 'strt_date', 'strt_hour','strt_day_of_week']]
vod09_1 = vod09[['subsr', 'vod_strip', 'ct_cl', 'genre_of_ct_cl', 'use_tms',
        'disp_rtm_sec', 'strt_date', 'strt_hour','strt_day_of_week']]

In [29]:
vod08_1['use_tms_per_vod'] = vod08_1['use_tms']/ vod08_1['disp_rtm_sec']
vod09_1['use_tms_per_vod'] = vod09_1['use_tms']/ vod09_1['disp_rtm_sec']

<ipython-input-29-fd12bdf125c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod08_1['use_tms_per_vod'] = vod08_1['use_tms']/ vod08_1['disp_rtm_sec']
<ipython-input-29-fd12bdf125c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod09_1['use_tms_per_vod'] = vod09_1['use_tms']/ vod09_1['disp_rtm_sec']


In [30]:
vod08_1

,subsr,vod_strip,ct_cl,genre_of_ct_cl,use_tms,disp_rtm_sec,strt_date,strt_hour,strt_day_of_week,use_tms_per_vod
0,65941000,그것이알고싶다,TV 시사/교양,기타,4800,4800,2023-08-12,16,Saturday,1.000000
1,66873000,그것이알고싶다,TV 시사/교양,기타,4800,4800,2023-08-16,20,Wednesday,1.000000
2,66873000,그것이알고싶다,TV 시사/교양,기타,718,4800,2023-08-29,19,Tuesday,0.149583
3,61689000,그것이알고싶다,TV 시사/교양,기타,4800,4800,2023-08-13,13,Sunday,1.000000
4,61619000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,4183,4380,2023-08-04,9,Friday,0.955023
...,...,...,...,...,...,...,...,...,...,...
5610,66317000,이파라파냐무냐무,키즈,학습,13,720,2023-08-10,22,Thursday,0.018056
5611,66317000,이파라파냐무냐무,키즈,학습,537,720,2023-08-10,22,Thursday,0.745833
5612,66317000,이파라파냐무냐무,키즈,학습,311,720,2023-08-10,22,Thursday,0.431944
5613,66317000,이파라파냐무냐무,키즈,학습,19,720,2023-08-10,22,Thursday,0.026389


In [31]:
# VOD 일별 합
# subsr,'vod_strip', strt_date 별로 use_tms_per_vod 합 컬럼 추가 (11/23)

vod08_1['total_use_tms'] = vod08_1.groupby(['subsr', 'vod_strip', 'strt_date'])['use_tms_per_vod'].transform('sum')
vod08_1 = vod08_1.drop_duplicates(subset=['subsr', 'vod_strip', 'strt_date'])

vod09_1['total_use_tms'] = vod09_1.groupby(['subsr', 'vod_strip', 'strt_date'])['use_tms_per_vod'].transform('sum')
vod09_1 = vod09_1.drop_duplicates(subset=['subsr', 'vod_strip', 'strt_date'])

vod09_1

<ipython-input-31-460d3aa5f6ea>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod08_1['total_use_tms'] = vod08_1.groupby(['subsr', 'vod_strip', 'strt_date'])['use_tms_per_vod'].transform('sum')
<ipython-input-31-460d3aa5f6ea>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod09_1['total_use_tms'] = vod09_1.groupby(['subsr', 'vod_strip', 'strt_date'])['use_tms_per_vod'].transform('sum')


,subsr,vod_strip,ct_cl,genre_of_ct_cl,use_tms,disp_rtm_sec,strt_date,strt_hour,strt_day_of_week,use_tms_per_vod,total_use_tms
0,64385000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,7,4260,2023-09-20,15,Wednesday,0.001643,1.001643
1,65819000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,4260,4260,2023-09-17,4,Sunday,1.000000,4.000000
2,66744000,인간극장,TV 시사/교양,기타,1920,1920,2023-09-29,14,Friday,1.000000,1.392708
3,66885000,인간극장,TV 시사/교양,기타,1980,1980,2023-09-13,10,Wednesday,1.000000,3.000000
4,62421000,과학수사대스모킹건,TV 시사/교양,기타,0,3660,2023-09-01,23,Friday,0.000000,0.250556
...,...,...,...,...,...,...,...,...,...,...,...
5036,66900000,핑크퐁자동차동화,키즈,학습,11,240,2023-09-04,12,Monday,0.045833,0.045833
5037,60169000,간질간질,키즈,학습,360,360,2023-09-14,21,Thursday,1.000000,1.000000
5038,66317000,알사탕,키즈,학습,480,480,2023-09-03,22,Sunday,1.000000,1.416667
5040,66900000,월간아기상어,키즈,학습,46,60,2023-09-04,12,Monday,0.766667,0.766667


In [32]:
# cont08_1.groupby(['subsr','cont_log_date','cont_strip']).count()
# cont09_1.groupby(['subsr','cont_log_date','cont_strip']).count()
# vod08_1.groupby(['subsr','strt_date','vod_strip']).count()

In [33]:
# 8월 데이터
# cont08_1을 groupby하여 count 계산
cont_count = cont08_1.groupby(['subsr', 'cont_log_date', 'cont_strip']).size().reset_index(name='cont_count')

# vod08_1과 cont_count을 병합하여 cont_count 열 추가
vod08_1 = pd.merge(vod08_1, cont_count, how='left', left_on=['subsr', 'strt_date', 'vod_strip'],
                   right_on=['subsr', 'cont_log_date', 'cont_strip'])

# NaN 값을 0으로 채우기 (count가 없는 경우)
vod08_1['cont_count'] = vod08_1['cont_count'].fillna(0).astype(int)

# 불필요한 컬럼 제거
vod08_1 = vod08_1.drop(['cont_log_date', 'cont_strip'], axis=1)

In [34]:
# 9월 데이터 반영
cont_count = cont09_1.groupby(['subsr', 'cont_log_date', 'cont_strip']).size().reset_index(name='cont_count')

vod09_1 = pd.merge(vod09_1, cont_count, how='left', left_on=['subsr', 'strt_date', 'vod_strip'],
                   right_on=['subsr', 'cont_log_date', 'cont_strip'])

# NaN 값을 0으로 채우기 (count가 없는 경우)
vod09_1['cont_count'] = vod09_1['cont_count'].fillna(0).astype(int)

# 불필요한 컬럼 제거
vod09_1 = vod09_1.drop(['cont_log_date', 'cont_strip'], axis=1)

In [35]:
# 5분 이하로 본 것은 이상치로 간주하고 삭제

# vod08_1 = vod08_1[vod08_1['total_use_tms']>300]
# vod09_1 = vod09_1[vod09_1['total_use_tms']>300]

In [36]:
vod09_1

,subsr,vod_strip,ct_cl,genre_of_ct_cl,use_tms,disp_rtm_sec,strt_date,strt_hour,strt_day_of_week,use_tms_per_vod,total_use_tms,cont_count
0,64385000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,7,4260,2023-09-20,15,Wednesday,0.001643,1.001643,3
1,65819000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,4260,4260,2023-09-17,4,Sunday,1.000000,4.000000,1
2,66744000,인간극장,TV 시사/교양,기타,1920,1920,2023-09-29,14,Friday,1.000000,1.392708,2
3,66885000,인간극장,TV 시사/교양,기타,1980,1980,2023-09-13,10,Wednesday,1.000000,3.000000,6
4,62421000,과학수사대스모킹건,TV 시사/교양,기타,0,3660,2023-09-01,23,Friday,0.000000,0.250556,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2165,66900000,핑크퐁자동차동화,키즈,학습,11,240,2023-09-04,12,Monday,0.045833,0.045833,0
2166,60169000,간질간질,키즈,학습,360,360,2023-09-14,21,Thursday,1.000000,1.000000,0
2167,66317000,알사탕,키즈,학습,480,480,2023-09-03,22,Sunday,1.000000,1.416667,0
2168,66900000,월간아기상어,키즈,학습,46,60,2023-09-04,12,Monday,0.766667,0.766667,0


#### 모델 적용

In [37]:
vod_combined = pd.concat([vod08_1, vod09_1], axis=0, ignore_index=True)
vod_combined

,subsr,vod_strip,ct_cl,genre_of_ct_cl,use_tms,disp_rtm_sec,strt_date,strt_hour,strt_day_of_week,use_tms_per_vod,total_use_tms,cont_count
0,65941000,그것이알고싶다,TV 시사/교양,기타,4800,4800,2023-08-12,16,Saturday,1.000000,2.000000,2
1,66873000,그것이알고싶다,TV 시사/교양,기타,4800,4800,2023-08-16,20,Wednesday,1.000000,1.359453,2
2,66873000,그것이알고싶다,TV 시사/교양,기타,718,4800,2023-08-29,19,Tuesday,0.149583,2.763016,3
3,61689000,그것이알고싶다,TV 시사/교양,기타,4800,4800,2023-08-13,13,Sunday,1.000000,1.000000,2
4,61619000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,4183,4380,2023-08-04,9,Friday,0.955023,0.955023,2
...,...,...,...,...,...,...,...,...,...,...,...,...
4610,66900000,핑크퐁자동차동화,키즈,학습,11,240,2023-09-04,12,Monday,0.045833,0.045833,0
4611,60169000,간질간질,키즈,학습,360,360,2023-09-14,21,Thursday,1.000000,1.000000,0
4612,66317000,알사탕,키즈,학습,480,480,2023-09-03,22,Sunday,1.000000,1.416667,0
4613,66900000,월간아기상어,키즈,학습,46,60,2023-09-04,12,Monday,0.766667,0.766667,0


In [38]:
vod_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4615 entries, 0 to 4614
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   subsr             4615 non-null   int64         
 1   vod_strip         4615 non-null   object        
 2   ct_cl             4615 non-null   object        
 3   genre_of_ct_cl    4615 non-null   object        
 4   use_tms           4615 non-null   int64         
 5   disp_rtm_sec      4615 non-null   int64         
 6   strt_date         4615 non-null   datetime64[ns]
 7   strt_hour         4615 non-null   int64         
 8   strt_day_of_week  4615 non-null   object        
 9   use_tms_per_vod   4535 non-null   float64       
 10  total_use_tms     4615 non-null   float64       
 11  cont_count        4615 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 432.8+ KB


In [39]:
 vod_combined['subsr'].values

array([65941000, 66873000, 66873000, ..., 66317000, 66900000, 60169000])

In [40]:
# FM 모델 구현하고 VOD 시청시간 ('total_use_tms)를 예측하는 모델
# => 분류 모델이 아니기에 평가지표를 recall,precision 가 아닌 MSE를 사용함

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler

# 데이터 로드
# vod_df = vod08_1.copy()
vod_df = vod_combined.copy()

# 'subsr'를 index로 설정
vod_df.set_index('subsr', inplace=True)

# Object 컬럼에 대해 One-hot Encoding
categorical_columns = ['vod_strip', 'ct_cl', 'genre_of_ct_cl', 'strt_day_of_week']
encoders = []
encoded_features = []

for col in categorical_columns:
    one_hot_encoder = OneHotEncoder(sparse=False)
    encoded_column = one_hot_encoder.fit_transform(vod_df[col].values.reshape(-1, 1))
    encoded_features.append(encoded_column)

# 정규화가 필요한 열들
columns_to_normalize = ['disp_rtm_sec', 'strt_hour', 'cont_count', 'total_use_tms']
minmax_scaler = MinMaxScaler()
vod_df[columns_to_normalize] = minmax_scaler.fit_transform(vod_df[columns_to_normalize])

# encoding 및 정규화된 특성 결합
X = np.hstack(encoded_features + [vod_df[columns_to_normalize]])
print(X)

# target 설정
y = vod_df['total_use_tms'].astype(float)

# 데이터를 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

[[0.         0.         0.         ... 0.69565217 0.03846154 0.09090909]
 [0.         0.         0.         ... 0.86956522 0.03846154 0.06179331]
 [0.         0.         0.         ... 0.82608696 0.05769231 0.12559164]
 ...
 [0.         0.         0.         ... 0.95652174 0.         0.06439394]
 [0.         0.         0.         ... 0.52173913 0.         0.03484848]
 [0.         0.         0.         ... 0.91304348 0.         0.04404762]]


### 평가 결과
- MSE : 3.74 (러닝타임 별 시청시간 고려 전)
- MSE : 3.2611(러닝타임 별 시청시간 고려 적용)

In [41]:
# FM 모델 정의
class FM(tf.keras.Model):
    def __init__(self):
        super(FM, self).__init__()

        # 모델의 파라미터 정의
        self.w_0 = tf.Variable([0.0]) #선형 항의 편향
        self.w = tf.Variable(tf.zeros(X.shape[1])) # 선형 항의 가중치
        self.V = tf.Variable(tf.random.normal(shape=(X.shape[1], 10)))

    # Call Method : 입력으로 받은 특성에 대해 선형 항과 상호 작용 항을 계산하여 최종 예측값을 반환
    # 입력 특성과 'self.V' 간의 행렬 곱 계산
    def call(self, inputs):
        linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)

        interactions = 0.5 * tf.reduce_sum(
            tf.math.pow(tf.matmul(inputs, self.V), 2)
            - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.V, 2)),
            1,
            keepdims=False
        )

        y_hat = self.w_0 + linear_terms + interactions

        return y_hat

# 모델 초기화
model = FM()

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 훈련
model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

# 모델 예측
y_pred = model.predict(X_test)

# 평균 제곱 오차(Mean Squared Error) 계산
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")

Epoch 1/10
462/462 [==============================] - 3s 3ms/step - loss: 42.3234 - val_loss: 29.6009
Epoch 2/10
462/462 [==============================] - 1s 2ms/step - loss: 24.3671 - val_loss: 19.8460
Epoch 3/10
462/462 [==============================] - 1s 2ms/step - loss: 16.4417 - val_loss: 14.6154
Epoch 4/10
462/462 [==============================] - 1s 3ms/step - loss: 11.8227 - val_loss: 11.2515
Epoch 5/10
462/462 [==============================] - 1s 3ms/step - loss: 8.7646 - val_loss: 8.8635
Epoch 6/10
462/462 [==============================] - 1s 2ms/step - loss: 6.5963 - val_loss: 7.1076
Epoch 7/10
462/462 [==============================] - 1s 3ms/step - loss: 5.0120 - val_loss: 5.7640
Epoch 8/10
462/462 [==============================] - 1s 2ms/step - loss: 3.8346 - val_loss: 4.7191
Epoch 9/10
462/462 [==============================] - 1s 3ms/step - loss: 2.9534 - val_loss: 3.9084
Epoch 10/10
29/29 [==============================] - 0s 3ms/step
Mean Squared Error: 3.2611


#### SUBSR 별 예측 결과(MSE)

In [42]:
unique_subsr_values = vod_df.index.unique()

In [43]:
# 예측값과 실제값을 데이터프레임으로 변환
predictions_df = pd.DataFrame({
    'subsr': np.repeat(unique_subsr_values, X_test.shape[0] // len(unique_subsr_values)),
    'predicted_total_use_tms': y_pred.flatten()[:len(unique_subsr_values) * (X_test.shape[0] // len(unique_subsr_values))],
    'actual_total_use_tms': np.tile(y_test.values, X_test.shape[0] // len(unique_subsr_values))[:len(unique_subsr_values) * (X_test.shape[0] // len(unique_subsr_values))]
})

# 각 subsr에 대한 예측값에 대한 mse 출력
for subsr in unique_subsr_values:
    mse_subsr = mean_squared_error(predictions_df[predictions_df['subsr'] == subsr]['actual_total_use_tms'],
                                    predictions_df[predictions_df['subsr'] == subsr]['predicted_total_use_tms'])
    print(f"MSE for Subsr {subsr}: {mse_subsr:.4f}")


MSE for Subsr 65941000: 0.3145
MSE for Subsr 66873000: 1.9775
MSE for Subsr 61689000: 4.0260
MSE for Subsr 61619000: 0.1131
MSE for Subsr 64495000: 0.9803
MSE for Subsr 65260000: 6.6828
MSE for Subsr 66885000: 0.7724
MSE for Subsr 62701000: 2.8683
MSE for Subsr 65954000: 0.6257
MSE for Subsr 67055000: 0.5398
MSE for Subsr 64376000: 3.5933
MSE for Subsr 65696000: 1.3954
MSE for Subsr 64659000: 1.3481
MSE for Subsr 61683000: 3.9274
MSE for Subsr 64154000: 0.7131
MSE for Subsr 60984000: 1.7442
MSE for Subsr 64385000: 0.4869
MSE for Subsr 66163000: 16.1593
MSE for Subsr 66353000: 4.1242
MSE for Subsr 62421000: 0.8989
MSE for Subsr 65443000: 0.3097
MSE for Subsr 59900000: 0.5269
MSE for Subsr 60593000: 1.6803
MSE for Subsr 66309000: 1.3424
MSE for Subsr 64520000: 3.5188
MSE for Subsr 64536000: 0.5161
MSE for Subsr 64040000: 14.1492
MSE for Subsr 65819000: 3.6854
MSE for Subsr 66900000: 0.3902
MSE for Subsr 63030000: 8.9553
MSE for Subsr 65386000: 4.3393
MSE for Subsr 63419000: 9.0650
MSE fo